from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/My\ Drive/UROP/project/UROP_project/

# 478

In [2]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [29]:
files = utils.files_in_order('poses_compressed/squat')

X_train_names, X_test_names = train_test_split(files, test_size=0.5, random_state=21)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)

pprint(X_train_names)
pprint(y_train)
pprint(X_test_names)
pprint(y_test)

array(['squat_bad_8.npy', 'squat_good_5.npy', 'squat_good_16.npy',
       'squat_good_14.npy', 'squat_bad_4.npy', 'squat_good_6.npy',
       'squat_good_10.npy', 'squat_good_11.npy'], dtype='<U17')
array([0, 1, 1, 1, 0, 1, 1, 1])
array(['squat_good_17.npy', 'squat_good_8.npy', 'squat_good_15.npy',
       'squat_good_7.npy', 'squat_good_13.npy', 'squat_good_12.npy',
       'squat_bad_7.npy', 'squat_good_9.npy'], dtype='<U17')
array([1, 1, 1, 1, 1, 1, 0, 1])


In [30]:
def load_features(names, flag):
    output1 = [] # List of upper arm torso angles
    output2 = [] # List of upper_leg_under_leg_angle
    output3 = [] # List of under_leg_ground_angle
    
    for filename in names:
        if flag == 0: # flag 0 이면 기존 training, test 
            ps = load_ps('poses_compressed/squat/'+filename)
        else: # 1 flag 이면 새로 만든 값
            ps = load_ps('numpy/'+filename)
        
        poses = ps.poses

        right_present = [1 for pose in poses 
                if pose.rankle.exists and pose.rknee.exists and pose.rhip.exists]
        left_present = [1 for pose in poses
                if pose.lankle.exists and pose.lknee.exists and pose.lhip.exists]
        right_count = sum(right_present)
        left_count = sum(left_present)
        side = 'right' if right_count > left_count else 'left'

        if side == 'right':
            joints = [(pose.rankle, pose.rknee, pose.rhip, pose.neck) for pose in poses]
        else:
            joints = [(pose.lankle, pose.lknee, pose.lhip,pose.neck) for pose in poses]

        # filter out data points where a part does not exist
        joints = [joint for joint in joints if all(part.exists for part in joint)]
        
        torso_vecs = np.array([(joint[3].x - joint[2].x, joint[3].y - joint[2].y) for joint in joints])#몸통벡터
        upper_leg_vecs = np.array([(joint[2].x - joint[1].x, joint[2].y - joint[1].y) for joint in joints])#허벅지 벡터
        under_leg_vecs = np.array([(joint[1].x - joint[0].x, joint[1].y - joint[0].y) for joint in joints])#정강이 벡터
        ground_vecs=np.array([((joint[0].x)+4,0) for joint in joints])#x축과 평행한 벡터(땅벡터)

        torso_vecs = torso_vecs / np.expand_dims(np.linalg.norm(torso_vecs, axis=1), axis=1)
        upper_leg_vecs = upper_leg_vecs / np.expand_dims(np.linalg.norm(upper_leg_vecs, axis=1), axis=1)
        under_leg_vecs = under_leg_vecs / np.expand_dims(np.linalg.norm(under_leg_vecs, axis=1), axis=1)
        ground_vecs = ground_vecs / np.expand_dims(np.linalg.norm(under_leg_vecs, axis=1), axis=1)

        upper_leg_torso_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_leg_vecs, torso_vecs), axis=1), -1.0, 1.0)))
        upper_arm_torso_angle_filtered = medfilt(medfilt(upper_leg_torso_angle, 5), 5)
        
        upper_leg_under_leg_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_leg_vecs, under_leg_vecs), axis=1), -1.0, 1.0)))
        upper_leg_under_leg_angle_filtered = medfilt(medfilt(upper_leg_under_leg_angle, 5), 5)

        under_leg_ground_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(under_leg_vecs, ground_vecs), axis=1), -1.0, 1.0)))
        under_leg_ground_angle_filtered = medfilt(medfilt(under_leg_ground_angle, 5), 5)

        output1.append(upper_arm_torso_angle_filtered.tolist())
        output2.append(upper_leg_under_leg_angle_filtered.tolist())
        output3.append(under_leg_ground_angle_filtered.tolist())
     
        
    return output1, output2, output3

X_train_1, X_train_2, X_train_3 = load_features(X_train_names, 0)
X_test_1, X_test_2, X_test_3 = load_features(X_test_names, 0)
print(X_train_names)
print(len(X_train_1))


'''
X_train_1=load_features(X_train_names,0)
X_train_2=load_features(X_train_names,0)
X_test_1=load_features(X_test_names, 0)
X_test_2=load_features(X_test_names, 0)
X_new_names = ['test.npy']
X_new_1= load_features(X_new_names, 1)
X_new_2= load_features(X_new_names, 1)
'''




['squat_bad_8.npy' 'squat_good_5.npy' 'squat_good_16.npy'
 'squat_good_14.npy' 'squat_bad_4.npy' 'squat_good_6.npy'
 'squat_good_10.npy' 'squat_good_11.npy']
8


"\nX_train_1=load_features(X_train_names,0)\nX_train_2=load_features(X_train_names,0)\nX_test_1=load_features(X_test_names, 0)\nX_test_2=load_features(X_test_names, 0)\nX_new_names = ['test.npy']\nX_new_1= load_features(X_new_names, 1)\nX_new_2= load_features(X_new_names, 1)\n"

In [31]:
# scaling 후에 정해진 위치로 shift
def normalization(pose) :
    # lenght normalization
    # torso 길이가 1이 되도록 scale param을 구하고 resize
    fixed_torso_legnth = 1 # tosro 길이를 1로 설정
    
    torso_lengths = np.array([dist(pose.neck, pose.lhip)] +
                                 [dist(pose.neck, pose.rhip)])
    mean_torso = np.mean(torso_lengths)
    scale = fixed_torso_legnth / mean_torso
    
    #print("\n\nScale")
    #print(scale)
    
    # shift
    # 2, 2로 맞추기
    set_x, set_y = 2, 2
    x, y = pose.neck.x*scale, pose.neck.y*scale
    
    shift_x = ((set_x-x)*(set_x-x))**0.5
    shift_y = ((set_y-y)*(set_y-y))**0.5
    
    if set_x < x : 
        shift_x = -shift_x
    if set_y < y : 
        shift_y = -shift_y
    
    #print("SHIFT")
    #print(shift_x,shift_y)
    #print(x + shift_x, y + shift_y)

    return scale, shift_x, shift_y

def dist(part1, part2):
    return np.sqrt(np.square(part1.x - part2.x) + np.square(part1.y - part2.y))


In [32]:
import matplotlib.pyplot as plt

def plot_keypoints(x, y) :
    plt.xlim([0,5])
    plt.ylim([0,5])
    plt.scatter(x, y)
    plt.show()

    plt.rcdefaults() 
    

In [33]:
def cal_vector_change(x_array, y_array) : 
    result = np.array([])
    
    for t in range(1, len(x_array)) :
        tmp = []
        v = np.square(x_array[t-1] - x_array[t])
        w = np.square(y_array[t-1] - y_array[t])

        result = np.append(result, np.sum(np.sqrt(v+w)))
 
    #print(result.shape)
    return result

In [34]:
def calculate_change_keypoints(names, flag, mode='right'):
    # 촬영 방향
    # 오른쪽에서 촬영(오른쪽 관절만 관여)
    if mode == 'right' :
        PART_NAMES = ['neck', 'rshoulder', 'relbow', 'rwrist', 'rhip', 'rknee', 'rankle']
    

    # 왼쪽에서 촬영(왼쪽 관절만 관여)
    elif mode == 'left' :
        PART_NAMES = ['neck', 'lshoulder', 'lelbow', 'lwrist', 'lhip', 'lknee', 'lankle']

    # 앞 또는 뒤에서 촬영(모든 관절이 관여)
    else :
        PART_NAMES = ['neck',  'rshoulder', 'relbow', 'rwrist', 'rhip', 'rknee', 'rankle',
                      'lshoulder', 'lelbow', 'lwrist', 'lhip', 'lknee', 'lankle']
    
    result = []
    for filename in names:
        output = []
        x_array = []
        y_array = []

        #tmp_np = np.load('poses_compressed/squat/'+filename)

        if flag == 0: # flag 0 이면 기존 training, test 
            ps = load_ps('poses_compressed/squat/'+filename)

        # (!! 수정필요 - 새로운 파일 돌릴 수 있도록)
        else: # 1 flag 이면 새로 만든 값 
            ps = load_ps('poses_compressed/squat/'+filename)
        
        poses = ps.poses
        
        for pose in poses : 
            x = []
            y = []
            
            scale, shift_x, shift_y = normalization(pose)

            for part in PART_NAMES :
                # if getattr(pose, part).x < 0 : continue
                #print(part)
                #print(getattr(pose, part).x * scale + shift_x, getattr(pose, part).y * scale + shift_y)
                x.append(getattr(pose, part).x * scale + shift_x)
                y.append(getattr(pose, part).y * scale + shift_y)
                  
            x_array.append(x)
            y_array.append(y)
            #plot_keypoints(x, y)
            #break
            
        x_array = np.array(x_array)
        y_array = np.array(y_array)

        #print(x_array.shape)
        #output = np.array(output)
        #output = output.transpose(1, 0, 2)  
      
        output = cal_vector_change(x_array, y_array)

        result.append(output)
 
    # print("LEN")
    # for l in result : 
    #  print(len(l))
    # print("END")
    result = np.array(result)

    # print("calculate")
    # print(result.shape)
    return result

print(X_train_names[0])
calculate_change_keypoints([X_train_names[0]], 0, 'right')
#calculate_change_keypoints(X_train_names, 0, 'right')

squat_bad_8.npy


array([[0.83301186, 0.49884838, 0.46586218, 0.30067925, 0.25277905,
        0.29023592, 0.21320309, 0.18845771, 0.27356097, 0.25596851,
        0.25021349, 0.28407496, 0.4016232 , 0.09631213, 0.08578329,
        0.29350977, 0.17107942, 0.15610167, 0.1672464 , 0.11627831,
        0.13827477, 0.24964049, 0.20762738, 0.28657717, 0.43425388,
        0.38632344, 0.22475158, 0.17199153, 0.31794127, 0.24217511,
        0.27734601, 0.61835204, 1.20635302, 0.45910641]])

In [35]:
def KNN_with_keypoints(X_trains, X_test, X_1, X_2, X_3):
    predictions = []
    train_d_vectors = calculate_change_keypoints(X_train_names, 0)
    test_d_vector   = calculate_change_keypoints([X_test], 1)
    
    # Store the average distance to good and bad training examples
    good, bad = [[] for i in range(2)]
    f1_good, f1_bad, f2_good, f2_bad, f3_good, f3_bad = [[] for i in range(6)]

    for idx, train_d_vector in enumerate(train_d_vectors): # X_test_names
        # Store the average distance to good and bad training examples
        dist = utils.DTWDistance(train_d_vector, test_d_vector[0])
        dist1 = utils.DTWDistance(X_train_1[i], X_1)
        dist2 = utils.DTWDistance(X_train_2[i], X_2)
        dist3 = utils.DTWDistance(X_train_3[i], X_3)
            
        if y_train[idx]:
            good.append(dist)
            f1_good.append(dist1)
            f2_good.append(dist2)
            f3_good.append(dist3)
            
        else:
            bad.append(dist)
            f1_bad.append(dist1)
            f2_bad.append(dist2)
            f3_bad.append(dist3)
              
        # if len(good) == 0 : good = [0]
        # if len(bad) == 0 : bad = [0]
        
    # 평가시 Train에 good, bad를 모두 넣어줘야함 
    good_score = np.mean(good) + np.mean(f1_good) + np.mean(f2_good) + np.mean(f3_good)
    bad_score  = np.mean(bad) + np.mean(f1_bad) + np.mean(f2_bad) + np.mean(f3_bad)
    
    print(good_score, bad_score)
    
    #print(good_score, bad_score)
    # dist가 크면 유사도가 적다
    # bad score가 크면 good
    if good_score < bad_score:
        return 1
    else:
        return 0
    
result = []
for i in range(len(X_test_names)) :
    print(X_test_names[i])
    test_label = KNN_with_keypoints(X_train_names, X_test_names[i], X_test_1[i], X_test_2[i], X_test_3[i]) # 기존 test data 
    #print(test_label)
    
    result.append([X_test_names[i], test_label])

squat_good_17.npy
458.12374967122435 458.68038070089176
squat_good_8.npy
90.19801411140207 90.77855992820905
squat_good_15.npy


/Users/imjungsun/.pyenv/versions/urop_env/lib/python3.6/site-packages/ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


97.25489535024144 97.68067604509845
squat_good_7.npy
89.85709169215558 90.25674482101637
squat_good_13.npy
544.3654779566743 544.9764932085488
squat_good_12.npy
517.9444594523251 517.0766040243949
squat_bad_7.npy
184.99402829139308 185.16690046520063
squat_good_9.npy
93.72199105280822 93.98097572800174


In [36]:
print("## TRAIN SET ##")
for i in range(len(X_train_names)) : 
    print(X_train_names[i], end = " ")
    if y_train[i] == 1 :
        print("Good")
    else :
        print("Bad")
        
        
print("\n## TEST RESULT ##")
for r in result : 
    ori_result = r[0].split("_")[1]
    
    print(r[0], end = " ")
    if r[1] == 1 :
        print("Good", end = " ")
    else :
        print("Bad", end = " ")

    if ori_result == 'good' and r[1] == 1 : 
        print("Right")  
    
    elif ori_result == 'bad' and r[1] == 0 : 
        print("Right")
        
    else : print("Wrong")

## TRAIN SET ##
squat_bad_8.npy Bad
squat_good_5.npy Good
squat_good_16.npy Good
squat_good_14.npy Good
squat_bad_4.npy Bad
squat_good_6.npy Good
squat_good_10.npy Good
squat_good_11.npy Good

## TEST RESULT ##
squat_good_17.npy Good Right
squat_good_8.npy Good Right
squat_good_15.npy Good Right
squat_good_7.npy Good Right
squat_good_13.npy Good Right
squat_good_12.npy Bad Wrong
squat_bad_7.npy Good Wrong
squat_good_9.npy Good Right
